# Hidden Markov Model for Human Activity RecognitionThis notebook implements data preprocessing, feature extraction, model training, and evaluation using Hidden Markov Models.

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.fft import fft
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score
from hmmlearn import hmm
import glob, os

In [ ]:
# Load data from /data folder
data_files = glob.glob('data/*.csv')
dfs = []
for file in data_files:
    df = pd.read_csv(file)
    df['activity'] = os.path.basename(file).split('_')[0].lower()
    dfs.append(df)
raw_data = pd.concat(dfs, ignore_index=True)
raw_data.head()

In [ ]:
# Feature extraction function
def extract_features(df):
    feats = {}
    for axis in ['x', 'y', 'z']:
        feats[f'mean_{axis}'] = df[axis].mean()
        feats[f'std_{axis}'] = df[axis].std()
    sma = np.mean(np.abs(df[['x','y','z']])).sum()
    feats['sma'] = sma
    freq = np.abs(fft(df['x']))
    feats['dominant_freq'] = np.argmax(freq)
    return feats

In [ ]:
# Create feature dataset
feature_rows = []
for file in data_files:
    df = pd.read_csv(file)
    feats = extract_features(df)
    feats['activity'] = os.path.basename(file).split('_')[0].lower()
    feature_rows.append(feats)
features_df = pd.DataFrame(feature_rows)
features_df.head()

In [ ]:
# Normalize and encode
scaler = StandardScaler()
X = scaler.fit_transform(features_df.drop('activity', axis=1))
y = features_df['activity'].astype('category').cat.codes
activity_map = dict(enumerate(features_df['activity'].astype('category').cat.categories))
X[:5], y[:5]

In [ ]:
# Train HMM
model = hmm.GaussianHMM(n_components=len(activity_map), covariance_type='diag', n_iter=100)
model.fit(X)
predicted_states = model.predict(X)
decoded_labels = [activity_map[s] for s in predicted_states]
pd.DataFrame({'True': features_df['activity'], 'Predicted': decoded_labels}).head()

In [ ]:
# Evaluate performance
cm = confusion_matrix(features_df['activity'], decoded_labels, labels=activity_map.values())
sns.heatmap(cm, annot=True, fmt='d', xticklabels=activity_map.values(), yticklabels=activity_map.values())
plt.title('Confusion Matrix')
plt.show()
acc = accuracy_score(features_df['activity'], decoded_labels)
print(f'Overall Accuracy: {acc:.2f}')